<a href="https://colab.research.google.com/github/IvanOlvera25/2022-1-Comp-I/blob/main/AnalisisSentimental.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Método para limpiar el texto y realizar el sentiment analysis
1. Importamos nuestras librerias 

In [2]:
import tweepy
from textblob import TextBlob
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt 
plt.style.use('fivethirtyeight')


2. Importamos nuestra base de datos como un csv



In [ ]:
from google.colab import files
files.upload()



In [ ]:
dta = pd.read_csv("/content/2019Tweets.csv")
dta.head()


Ahora crearemos un subset con nomás las columnas de "Date" y "Body"

In [8]:
dta2 = dta[["Date", "body"]]

In [9]:
dta2.head(3)

,Date,body
0,2019-01-02,$AMZN $AAPL $BA $MA $SPY Live stream Tonigh...
1,2019-01-02,$AMZN $SPY $BA $AAPL todays action in the mark...
2,2019-01-02,Inquiring minds want to know. Is @jlebenthal ...


Ahora que ya leímos la base de datos vamos a crear una función que limpie el texto

In [5]:
def cleanTwt(twt):
  twt = re.sub('#apple', 'apple', twt)  
  twt = re.sub('#Apple', 'Apple', twt) 
  twt = re.sub('#[A-Za-z0-9]+', '', twt)
  twt = re.sub('\\n', '', twt)
  twt = re.sub('https?:\/\/\S+', '', twt)
  return twt

Aplicamos la función a nuestro subset a la columna de "body" y creamos una nueva columna con el texto limpio

In [10]:
dta2['Cleaned_Tweets'] = dta2['body'].apply(cleanTwt)
dta2.head(3)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Date,body,Cleaned_Tweets
0,2019-01-02,$AMZN $AAPL $BA $MA $SPY Live stream Tonigh...,$AMZN $AAPL $BA $MA $SPY Live stream Tonigh...
1,2019-01-02,$AMZN $SPY $BA $AAPL todays action in the mark...,$AMZN $SPY $BA $AAPL todays action in the mark...
2,2019-01-02,Inquiring minds want to know. Is @jlebenthal ...,Inquiring minds want to know. Is @jlebenthal ...


Creamos un nuevo subset con solo la fecha y el texto limpio

In [11]:
dta3 = dta2[["Date", "Cleaned_Tweets"]]
dta.head(3)

,Unnamed: 0,tweet_id,writer,Date,body,comment_num,retweet_num,like_num
0,1,1.080492e+18,Sandro_power,2019-01-02,$AMZN $AAPL $BA $MA $SPY Live stream Tonigh...,12,54,64
1,2,1.080581e+18,Sandro_power,2019-01-02,$AMZN $SPY $BA $AAPL todays action in the mark...,6,52,65
2,3,1.080583e+18,ScottWapnerCNBC,2019-01-02,Inquiring minds want to know. Is @jlebenthal ...,23,4,61


Ahora crearemos las funciones para asignar la polaridad del sentimiento (donde -1 es muy malo y 1 muy bueno) y la subjetividad. Y posteriormente, creamos 2 nuevas columnas con el nuevo puntaje.

In [12]:
def getSubjectivity(twt):
  return TextBlob(twt).sentiment.subjectivity

def getPolarity(twt):
  return TextBlob(twt).sentiment.polarity

dta3['Subjectivity'] = dta3['Cleaned_Tweets'].apply(getSubjectivity)
dta3['Polarity'] = dta3['Cleaned_Tweets'].apply(getPolarity)

dta3.head(5)

,Date,Cleaned_Tweets,Subjectivity,Polarity
0,2019-01-02,$AMZN $AAPL $BA $MA $SPY Live stream Tonigh...,0.402652,0.046970
1,2019-01-02,$AMZN $SPY $BA $AAPL todays action in the mark...,0.335714,0.333766
2,2019-01-02,Inquiring minds want to know. Is @jlebenthal ...,0.000000,0.000000
3,2019-01-02,Apple shares down 8 percent in extended trade ...,0.288889,-0.155556
4,2019-01-02,Lessons/conclusions from $AAPL:* Be skeptical ...,0.750000,0.050000


Ahora crearemos una función para asignar un sentimiento a los tweets. Donde si la polaridad es >0 es negativo, = 0 es neutro y <0 es positivo.

In [13]:
def getSentiment(score):
  if score < 0:
    return 'Negative'
  elif score == 0:
    return 'Neutral'
  else:
    return 'Positive'

Posteriormente creamos una nueva columna con el sentimiento de cada tweet.

In [15]:
dta3['Sentiment'] = dta3['Polarity'].apply(getSentiment)
dta3.head(5)

,Date,Cleaned_Tweets,Subjectivity,Polarity,Sentiment
0,2019-01-02,$AMZN $AAPL $BA $MA $SPY Live stream Tonigh...,0.402652,0.046970,Positive
1,2019-01-02,$AMZN $SPY $BA $AAPL todays action in the mark...,0.335714,0.333766,Positive
2,2019-01-02,Inquiring minds want to know. Is @jlebenthal ...,0.000000,0.000000,Neutral
3,2019-01-02,Apple shares down 8 percent in extended trade ...,0.288889,-0.155556,Negative
4,2019-01-02,Lessons/conclusions from $AAPL:* Be skeptical ...,0.750000,0.050000,Positive
